In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
import pickle

# Load dataset
data = pd.read_csv('dataset.csv')

# Data cleaning function
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower().strip()
    return text

# Apply cleaning
data['ReviewContent'] = data['ReviewContent'].apply(clean_text)

# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['ReviewContent'])
sequences = tokenizer.texts_to_sequences(data['ReviewContent'])
padded_sequences = pad_sequences(sequences, maxlen=200)

# Encode labels
label_encoder = LabelEncoder()
data['Sentiment'] = label_encoder.fit_transform(data['Sentiment'])

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['Sentiment'], test_size=0.2, random_state=42)

# Model building
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Save the entire model
model.save('bilstm_model.h5')

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the label encoder
with open('label_encoder.pkl', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)


Epoch 1/10
67/67 [==============================] - 47s 614ms/step - loss: 0.9408 - accuracy: 0.7016 - val_loss: 0.9371 - val_accuracy: 0.6906
Epoch 2/10
67/67 [==============================] - 44s 666ms/step - loss: 0.8029 - accuracy: 0.7200 - val_loss: 0.8989 - val_accuracy: 0.7019
Epoch 3/10
67/67 [==============================] - 39s 589ms/step - loss: 0.6389 - accuracy: 0.7753 - val_loss: 1.0362 - val_accuracy: 0.6906
Epoch 4/10
67/67 [==============================] - 38s 568ms/step - loss: 0.5172 - accuracy: 0.8215 - val_loss: 1.2266 - val_accuracy: 0.6811
Epoch 5/10
67/67 [==============================] - 40s 594ms/step - loss: 0.4518 - accuracy: 0.8470 - val_loss: 1.2237 - val_accuracy: 0.6491
Epoch 6/10
67/67 [==============================] - 39s 589ms/step - loss: 0.3551 - accuracy: 0.8791 - val_loss: 1.3168 - val_accuracy: 0.6321
Epoch 7/10
67/67 [==============================] - 39s 578ms/step - loss: 0.2848 - accuracy: 0.9084 - val_loss: 1.4930 - val_accuracy: 0.6434

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pickle

def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and numbers
    text = text.lower().strip()  # Convert to lower case and strip whitespaces
    return text

model = load_model('bilstm_model.h5')

with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)


with open('label_encoder.pkl', 'rb') as handle:
    label_encoder = pickle.load(handle)

def preprocess_input(text, tokenizer, max_length=200):
    text = clean_text(text)
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    return padded_sequence

input_text = "said when I ordered that it would arrive on January 5th, then the next day I check and see that it will arrive on January 9th"

preprocessed_input = preprocess_input(input_text, tokenizer)

prediction = model.predict(preprocessed_input)
predicted_label = np.argmax(prediction, axis=1)
predicted_sentiment = label_encoder.inverse_transform(predicted_label)

print(f'Predicted Sentiment: {predicted_sentiment[0]}')


1/1 [==============================] - 56s 56s/step
Predicted Sentiment: future expectation
